# Rejection-based EEG Signal Classification

## Import Libraries

In [2]:
import warnings

import matplotlib.pyplot as plt
import moabb
import numpy as np
import pandas as pd
import seaborn as sns
from mne.decoding import CSP
from moabb.datasets import BNCI2014_001, Cho2017, Lee2019_MI
from moabb.evaluations import (
    CrossSessionEvaluation,
    CrossSubjectEvaluation,
    WithinSessionEvaluation,
)
from moabb.paradigms import LeftRightImagery, MotorImagery
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline

moabb.set_log_level("info")
warnings.filterwarnings("ignore")
moabb.set_log_level("info")
warnings.filterwarnings("ignore")

## Instantiate Datasets

In [20]:
dataset_BNCI2014_001 = BNCI2014_001()
dataset_BNCI2014_001.subject_list = dataset_BNCI2014_001.subject_list[:1]

# datasets = [dataset_BNCI2014_001]

In [3]:
datasets = [BNCI2014_001(), Cho2017(), Lee2019_MI()]

An example of an EEG record obtained under the `mne` format, stored in a dictionary of sessions and runs.

In [28]:
sessions = dataset_BNCI2014_001.get_data()

In [22]:
sessions

{1: {'0train': {'0': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '1': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '2': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '3': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '4': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '5': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>},
  '1test': {'0': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '1': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '2': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '3': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '4': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>,
   '5': <RawArray | 26 x 96735 (386.9 s), ~19.2 MB, data loaded>}}}

## Choose a Paradigm

An example of a paradigm.

In [23]:
print(dataset_BNCI2014_001.paradigm)

imagery


In [6]:
paradigm = LeftRightImagery()

In [7]:
paradigm.datasets

The metric depends on the paradigm and the number of classes used with it. See `paradigm.__doc__` for more details. In the `LeftRightImagery()` case there are 2 classes.

In [8]:
metric = paradigm.scoring
print(metric)

roc_auc


## Unpack Datasets

The data stored in the `sklearn` format.

### BNCI2014_001

In [34]:
X, y, meta = paradigm.get_data(dataset=datasets[0])

100%|█████████████████████████████████████| 42.5M/42.5M [00:00<00:00, 32.8GB/s]
SHA256 hash of downloaded file: 77387d3b669f4ed9a7c1dac4dcba4c2c40c8910bae20fb961bb7cf5a94912950
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
100%|█████████████████████████████████████| 44.4M/44.4M [00:00<00:00, 38.6GB/s]
SHA256 hash of downloaded file: 8b357470865610c28b2f1d351beac247a56a856f02b2859d650736eb2ef77808
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
100%|█████████████████████████████████████| 44.6M/44.6M [00:00<00:00, 38.4GB/s]
SHA256 hash of downloaded file: 4dc3be1b0d60279134d1220323c73c68cf73799339a7fb224087a3c560a9a7e2
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
100%|████████████████████████████████

In [35]:
print(X.shape, type(X))

(2592, 22, 1001) <class 'numpy.ndarray'>


In [36]:
print(y, "\n", type(y))
print("The number of labels:", len(y))

['right_hand' 'left_hand' 'left_hand' ... 'left_hand' 'right_hand'
 'left_hand'] 
 <class 'numpy.ndarray'>
The number of labels: 2592


In [37]:
print(meta)

      subject session run
0           1  0train   0
1           1  0train   0
2           1  0train   0
3           1  0train   0
4           1  0train   0
...       ...     ...  ..
2587        9   1test   5
2588        9   1test   5
2589        9   1test   5
2590        9   1test   5
2591        9   1test   5

[2592 rows x 3 columns]


In [38]:
print(np.unique(meta["session"]))

['0train' '1test']


### Cho2017

In [ ]:
X, y, meta = paradigm.get_data(dataset=datasets[1])

In [ ]:
print(X.shape, type(X))

In [ ]:
print(y, "\n", type(y))
print("The number of labels:", len(y))

In [ ]:
print(meta)

In [ ]:
print(np.unique(meta["session"]))

### Lee2019_MI

In [ ]:
X, y, meta = paradigm.get_data(dataset=datasets[2])

In [ ]:
print(X.shape, type(X))

In [ ]:
print(y, "\n", type(y))
print("The number of labels:", len(y))

In [ ]:
print(meta)

In [ ]:
print(np.unique(meta["session"]))

## Sample Size Estimation with Bootstrap

### BNCI2014_001

In [9]:
cov_matrices, labels, meta = paradigm.get_data(dataset=datasets[0])
print(cov_matrices.shape, labels.shape)

(2592, 22, 1001) (2592,)


In [ ]:
# Define the number of bootstrap samples.
num_bootstraps = 10

# Define parameters for logistic regression model.
solver = "lbfgs"
random_state = 42

# Define the range of sample sizes to test.
step_size = cov_matrices.shape[0] // 5
sample_sizes = range(step_size, cov_matrices.shape[0], step_size)

# Initialize dictionary to store ROC AUC scores for each sample size.
roc_auc_scores = {}

# Perform bootstrapping for each sample size.
for sample_size in sample_sizes:
    # Initialize an array to store accuracy scores for the current sample size.
    roc_auc_scores[sample_size] = []

    # Perform bootstrapping.
    for _ in range(num_bootstraps):
        # Generate a bootstrap sample by randomly sampling from the original dataset with replacement.
        bootstrap_indices = np.random.choice(
            len(cov_matrices), size=sample_size, replace=True
        )
        X_bootstrap = cov_matrices[bootstrap_indices]
        y_bootstrap = labels[bootstrap_indices]

        # Split the data into training and validation sets.
        X_train, X_val, y_train, y_val = train_test_split(
            X_bootstrap, y_bootstrap, test_size=0.2, random_state=random_state
        )

        # Fit logistic regression model on the training data.
        clf = make_pipeline(
            Covariances(),
            TangentSpace(),
            LogisticRegression(solver=solver, random_state=random_state),
        )
        clf.fit(X_train, y_train)

        # Predict probabilities on the validation set.
        y_val_pred_proba = clf.predict_proba(X_val)[:, 1]

        # Calculate the ROC AUC score and store it.
        roc_auc = roc_auc_score(y_val, y_val_pred_proba)
        roc_auc_scores[sample_size].append(roc_auc)

# Calculate the mean ROC AUC scores for each sample size.
mean_roc_auc_scores = {
    sample_size: np.mean(scores) for sample_size, scores in roc_auc_scores.items()
}

# Choose the sample size with the highest mean ROC AUC score.
optimal_sample_size = max(mean_roc_auc_scores, key=mean_roc_auc_scores.get)
max_mean_roc_auc = mean_roc_auc_scores[optimal_sample_size]

print("Optimal sample size:", optimal_sample_size)
print("Maximum mean ROC AUC score:", max_mean_roc_auc)

In [ ]:
# Extract the sample sizes and mean ROC AUC scores from the dictionary.
sample_sizes = list(mean_roc_auc_scores.keys())
mean_roc_auc_scores_values = list(mean_roc_auc_scores.values())

# Plot ROC AUC vs Sample Size.
plt.figure(figsize=(10, 6))
plt.plot(sample_sizes, mean_roc_auc_scores_values, marker="o", linestyle="-")
plt.title("ROC AUC vs Sample Size")
plt.xlabel("Sample Size")
plt.ylabel("Mean ROC AUC Score")
plt.grid(True)
plt.tight_layout()
plt.show()

## Create a Pipeline

In [ ]:
# pipelines = {}

# pipelines["RG+LR"] = make_pipeline(
#     Covariances(), TangentSpace(), LogisticRegression(solver="lbfgs")
# )

## Evaluate a Solution

In [ ]:
# # Evaluate for a specific number of training samples per class.
# data_size = dict(policy="per_class", value=np.array([5, 10, 30, 50]))
# # When the training data is sparse, perform more permutations than when we have a lot of data.
# n_perms = np.floor(np.geomspace(20, 2, len(data_size["value"]))).astype(int)

In [ ]:
# evaluation = WithinSessionEvaluation(
#     paradigm=paradigm,
#     datasets=datasets,
#     overwrite=True,
#     data_size=data_size,
#     n_perms=n_perms,
# )

In [ ]:
# results = evaluation.process(pipelines)

In [ ]:
# results.head(n=5)

In [ ]:
# results.to_csv("./results.csv")

In [ ]:
# results = pd.read_csv("./results.csv")

## Plot the Results

In [ ]:
# fig, ax = plt.subplots(facecolor="white", figsize=[8, 4])

# n_subs = len(BNCI2014_001_dataset.subject_list)

# if n_subs > 1:
#     r = results.groupby(["pipeline", "subject", "data_size"]).mean().reset_index()
# else:
#     r = results

# sns.pointplot(data=r, x="data_size", y="score", hue="pipeline", ax=ax, palette="Set1")

# errbar_meaning = "subjects" if n_subs > 1 else "permutations"
# title_str = f"Error bar shows Mean-CI across {errbar_meaning}"
# ax.set_xlabel("Amount of training samples")
# ax.set_ylabel("ROC AUC")
# ax.set_title(title_str)
# fig.tight_layout()
# plt.show()